This project detects keypoints on an image of a yoga posture and tries to classify it. OpenPose is used to detect the keypoints and we use XGBoost to classify.

Installing OpenPose. We download and build the OpenPose repo from git to this notebook.

In [ ]:
#Code to download openpose and build 
#install OpenPose
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # install new CMake
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose git
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies and libraries
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake -DBUILD_PYTHON=ON .. && make -j`nproc`
  

print("installation done")

In [ ]:
!cd /content/openpose/build/python/ && pwd

/content/openpose/build/python


Building the Python wrapper for OpenPose. 

In [ ]:
os.chdir('/content/openpose/build/python/')
os.getcwd( )

!sudo make install -j `nproc`

[ 97%] Built target openpose
[100%] Built target pyopenpose
Install the project...
-- Install configuration: "Release"
-- Up-to-date: /usr/local/python/pyopenpose.cpython-36m-x86_64-linux-gnu.so
-- Up-to-date: /usr/local/python/openpose
-- Up-to-date: /usr/local/python/openpose/CMakeFiles
-- Up-to-date: /usr/local/python/openpose/CMakeFiles/pyopenpose.dir
-- Up-to-date: /usr/local/python/openpose/pyopenpose.cpython-36m-x86_64-linux-gnu.so
-- Up-to-date: /usr/local/python/openpose
-- Up-to-date: /usr/local/python/openpose/__init__.py
-- Up-to-date: /usr/local/python/openpose/CMakeFiles
-- Up-to-date: /usr/local/python/openpose/CMakeFiles/pyopenpose.dir


Mounting the Google Drive where the dataset is placed. 

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/mnt


In [ ]:
import sys
import cv2

from sys import platform
import argparse
import time

from openpose import pyopenpose as op

from matplotlib import pyplot as plt


In [ ]:
print(myData.poseKeypoints)


new_dF = myData.poseKeypoints.reshape (-1,3)
new_dF = pd.DataFrame (new_dF, columns = ['X', 'Y', 'P'])

new_dF = new_dF.iloc[0:25]


print(type(myData.poseKeypoints))
print(myData.poseKeypoints.shape)
print(new_dF)

new_dF = new_dF.T

new_dF = new_dF.join(new_dF.shift(-1).add_prefix('y_'))
new_dF[1::2] = ''

new_dF=new_dF.drop(['Y','P'])
print(new_dF) 


[[[0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [1.8841925e+02 3.4100290e+02 3.0423397e-01]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [1.4583594e+02 3.4413120e+02 4.5311826e-01]
  [1.5412740e+02 3.4724411e+02 4.2625058e-01]
  [1.8944478e+02 3.4931631e+02 2.5

Pointing to the directory where the image files are stored. A JSON file with the keypoints is created for each of the images. 

In [ ]:
#/content/mnt/My Drive/Datasets/YogaPoses/bhujangasana

!cd openpose && ./build/examples/openpose/openpose.bin --image_dir ../mnt/My\ Drive/Datasets/YogaPoses/uttanasana --write_json ../Uthanasana/ --display 0 --render_pose 0

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 9.079009 seconds.


A samples JSON looks like: 

> {"version":1.3,"people":[{"person_id":[-1],"pose_keypoints_2d":[248.431,219.657,0.285197,263.262,175.994,0.196737,233.555,181.559,0.14297,0,0,0,0,0,0,287.387,175.994,0.093796,227.975,256.764,0.801577,159.283,305.035,0.780612,340.313,123.994,0.0544115,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,241.001,218.665,0.56429,0,0,0,240.042,202.92,0.787322,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"face_keypoints_2d":[],"hand_left_keypoints_2d":[],"hand_right_keypoints_2d":[],"pose_keypoints_3d":[],"face_keypoints_3d":[],"hand_left_keypoints_3d":[],"hand_right_keypoints_3d":[]},{"person_id":[-1],"pose_keypoints_2d":[0,0,0,0,0,0,275.32,197.334,0.0854624,190.85,227.97,0.559881,131.418,278.089,0.790055,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"face_keypoints_2d":[],"hand_left_keypoints_2d":[],"hand_right_keypoints_2d":[],"pose_keypoints_3d":[],"face_keypoints_3d":[],"hand_left_keypoints_3d":[],"hand_right_keypoints_3d":[]},{"person_id":[-1],"pose_keypoints_2d":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,399.74,40.4328,0.328281,383.026,45.0668,0.14178,0,0,0,0,0,0,414.609,39.5025,0.407715,477.716,149.983,0.49397,527.864,258.6,0.629395,0,0,0,0,0,0,0,0,0,0,0,0,499.091,278.09,0.37415,508.363,278.139,0.379254,537.125,265.12,0.554026,0,0,0,0,0,0,0,0,0],"face_keypoints_2d":[],"hand_left_keypoints_2d":[],"hand_right_keypoints_2d":[],"pose_keypoints_3d":[],"face_keypoints_3d":[],"hand_left_keypoints_3d":[],"hand_right_keypoints_3d":[]}]}



[https://marc.schulder.info/files/slides/2019_10_03_openpose_for_linguists.pdf

A detailed description of what the JSON means can be found in the above mentioned link. For our use-case, we need just the numbers in the *pose_keypoints_2d*. 

In [ ]:
#CODE FOR CLASSIFICATION

import os, json
import pickle
import multiprocessing
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix



In [ ]:

#Labels
pose_list = ["AdhoMukhaSwana","Bhujangasana","Uthanasana","chathuranga"]
# A dataframe to store all the JSONs.
Main_DataFrame = pd.DataFrame()
path_to_json_parent = '/content/mnt/My Drive/Datasets/YogaPoses/YogaposesJSON/content/'


In [ ]:
#Function to preprocess the JSON.

def preprocess_json(pose_name):

      path_to_json = path_to_json_parent+pose_name
      json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
      global Main_DataFrame



      # we need both the json and an index number so use enumerate()
      for index, js in enumerate(json_files):
        with open(os.path.join(path_to_json, js)) as json_file:
            json_text = json.load(json_file)
          
            #Extracting hte pose_keypoints_2d from the JSON and adding it to a Dataframe.

            try:
              json_text = np.array(json_text ['people'] [0] ['pose_keypoints_2d']). reshape (-1,3)
              #print(json_text1)
              df = pd.DataFrame (json_text, columns = ['X', 'Y', 'P'])
              df = df.T

              df = df.join(df.shift(-1).add_prefix('y_'))
              df[1::2] = ''

              df=df.drop(['Y','P'])
              print(df) 
              df['Label'] = pose_name


              Main_DataFrame = Main_DataFrame.append(df,ignore_index=True)
              #print("mainDataFrame") 
              #print(mainDataFrame) 
            except IndexError:
              json_text = 'null'

                





In [ ]:
#Run hte function for each of the label
for pose_name in pose_list:
  preprocess_json(pose_name)

Now we have to split our data into Training and Test data. Firstly, we would map our input as x and y co ordinates of 24 keypoints) and output as the label.

In [ ]:

# Spliting the points and the labels
X = Main_DataFrame.iloc[:, :-1].values  
y = Main_DataFrame.iloc[:, 50].values

In [ ]:
# And split the data into appropriate data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
class_names = list(set(y))
num_class = len(class_names)
cores = multiprocessing.cpu_count()

In [ ]:
#Creating the XGBoost classifier. # Gradient Boosting
clf = XGBClassifier(max_depth=6, 
                    learning_rate=0.01, 
                    n_estimators=500, 
                    objective='multi:softmax', 
                    n_jobs=cores, 
                    num_class=num_class)

In [ ]:
#Training the model
clf.fit(X_train, y_train)
preds = clf.predict(X_test)

In [ ]:
# Generating the confusion matrix and the model summary.

from sklearn.metrics import confusion_matrix, classification_report
conf_matrix = confusion_matrix(y_test, preds)
print(conf_matrix)

class_report = classification_report(y_test, preds)
print(class_report)

[[11  0  1  0]
 [ 0  8  1  0]
 [ 0  0  6  3]
 [ 0  2  2 16]]
                precision    recall  f1-score   support

AdhoMukhaSwana       1.00      0.92      0.96        12
  Bhujangasana       0.80      0.89      0.84         9
    Uthanasana       0.60      0.67      0.63         9
   chathuranga       0.84      0.80      0.82        20

      accuracy                           0.82        50
     macro avg       0.81      0.82      0.81        50
  weighted avg       0.83      0.82      0.82        50



In [ ]:
# Saving the model
filename = '/content/models/yoga_poses.sav'
pickle.dump(clf, open(filename, 'wb'))

array(['Bhujangasana', 'Bhujangasana', 'chathuranga', 'chathuranga',
       'chathuranga', 'chathuranga', 'chathuranga', 'Bhujangasana',
       'chathuranga', 'Bhujangasana', 'AdhoMukhaSwana', 'AdhoMukhaSwana',
       'Uthanasana', 'chathuranga', 'Uthanasana', 'Uthanasana',
       'chathuranga', 'AdhoMukhaSwana', 'chathuranga', 'chathuranga',
       'chathuranga', 'Bhujangasana', 'Bhujangasana', 'chathuranga',
       'Uthanasana', 'chathuranga', 'chathuranga', 'Bhujangasana',
       'chathuranga', 'AdhoMukhaSwana', 'AdhoMukhaSwana',
       'AdhoMukhaSwana', 'Uthanasana', 'AdhoMukhaSwana', 'chathuranga',
       'chathuranga', 'AdhoMukhaSwana', 'Uthanasana', 'Bhujangasana',
       'Bhujangasana', 'Bhujangasana', 'chathuranga', 'AdhoMukhaSwana',
       'Uthanasana', 'chathuranga', 'Bhujangasana', 'Bhujangasana',
       'Bhujangasana', 'Uthanasana', 'Uthanasana'], dtype=object)

In [ ]:
#Doing Inference on a sample image from the internet

sample_image =  "https://www.miencuentroconmigo.com.ar/wp-content/uploads/2018/03/perro-hacia-abajo.jpg"

import requests

img_data = requests.get(sample_image).content
with open('image_name.jpg', 'wb') as handler:
    handler.write(img_data)





In [ ]:
!cd openpose && ./build/examples/openpose/openpose.bin --image_dir ../openpose/build/python/ --write_json ../Uthanasana/ --display 0 --render_pose 0


Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 6.338830 seconds.


In [ ]:
      path_to_json = '/content/Uthanasana/'
      json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
      test_DataFrame = pd.DataFrame()



      # we need both the json and an index number so use enumerate()
      for index, js in enumerate(json_files):
        with open(os.path.join(path_to_json, js)) as json_file:
            json_text2 = json.load(json_file)
            print(json_text2)
            # here you need to know the layout of your json and each json has to have
            # the same structure (obviously not the structure I have here)

            try:
              json_text1 = np.array(json_text2 ['people'] [0] ['pose_keypoints_2d']). reshape (-1,3)
              #print(json_text1)
              df = pd.DataFrame (json_text1, columns = ['X', 'Y', 'P'])
              df = df.T

              df = df.join(df.shift(-1).add_prefix('y_'))
              df[1::2] = ''

              df=df.drop(['Y','P'])
              print(df) 
              df['Label'] = "DownwardDogs"


              test_DataFrame = test_DataFrame.append(df,ignore_index=True)
              #print("mainDataFrame") 
              #print(mainDataFrame) 
            except IndexError:
              json_text1 = 'null'


{'version': 1.3, 'people': [{'person_id': [-1], 'pose_keypoints_2d': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 895.942, 218.498, 0.11408, 1102.39, 468.557, 0.253852, 1271.01, 747.689, 0.660162, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1166.34, 779.578, 0.463474, 1189.58, 785.527, 0.399337, 1297.18, 765.154, 0.735113, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'face_keypoints_2d': [], 'hand_left_keypoints_2d': [], 'hand_right_keypoints_2d': [], 'pose_keypoints_3d': [], 'face_keypoints_3d': [], 'hand_left_keypoints_3d': [], 'hand_right_keypoints_3d': []}, {'person_id': [-1], 'pose_keypoints_2d': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 913.352, 221.504, 0.143163, 933.758, 218.539, 0.107179, 1090.77, 454.04, 0.118886, 1247.69, 736.057, 0.259366, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1151.69, 776.671, 0.435561, 1151.72, 770.843, 0.359482, 0, 0, 0], 'face_keypoi

In [ ]:
test_DataFrame
test_DataFrame = test_DataFrame.drop(['Label'],axis=1)
test_DataFrame


new_preds = clf.predict(test_DataFrame.values)
new_preds

array(['AdhoMukhaSwana'], dtype=object)

And it predicted correctly :)